In [15]:
import os
from glob import glob
import copy
from datetime import datetime
import distutils.dir_util
from shutil import copyfile
import PIL


from matplotlib import pyplot as plt
%matplotlib inline
import bcolz
import ujson as json
import numpy as np

from keras.utils.np_utils import to_categorical as one_hot

from keras.layers import Dense, Input, Lambda, BatchNormalization, Conv2D, ZeroPadding2D, MaxPooling2D, AveragePooling2D, Activation, Flatten, Dropout
from keras.models import Model
from keras.applications.resnet50 import ResNet50, identity_block, conv_block
from keras.applications.imagenet_utils import preprocess_input
from keras.regularizers import l2

from keras.optimizers import Adam
from keras import backend as K

from keras.callbacks import CSVLogger
from keras.preprocessing.image import ImageDataGenerator

from keras import __version__
print(__version__)

2.0.4


In [2]:
DATA_ROOT_DIR = "/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/"
TEST_STG1_DIR = DATA_ROOT_DIR + '/test_stg1/'
TEST_STG2_DIR = DATA_ROOT_DIR + '/test_stg2/'
RESULTS_DIR = DATA_ROOT_DIR + '/results/'

SAMPLE_ROOT_DIR = DATA_ROOT_DIR + "/sample/"
TRAIN_SAMPLE_DIR = SAMPLE_ROOT_DIR + '/train/'
VALID_SAMPLE_DIR = SAMPLE_ROOT_DIR + '/valid/'
TRAIN_DIR = DATA_ROOT_DIR + '/train/'
VALID_DIR = DATA_ROOT_DIR + '/valid/'

# 1. Prepare Data (create validation/sample sets, rearrange images by class dir)

In [4]:
%mkdir -p $VALID_DIR
%mkdir -p $RESULTS_DIR
%mkdir -p $TRAIN_SAMPLE_DIR
#%mkdir -p $VALID_SAMPLE_DIR # "OMITTED AS VALIDATION SIZE QUITE SMALL ANYWAY"
%mkdir -p $TEST_STG1_DIR/unknown
%mkdir -p $TEST_STG2_DIR/unknown

In [11]:
!unzip sample_submission_stg1.csv.zip
!unzip sample_submission_stg2.csv.zip
!unzip test_stg1.zip
!7za e test_stg2.7z
!unzip train.zip


7-Zip (A) [64] 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
p7zip Version 9.20 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,4 CPUs)

Processing archive: test_stg2.7z

Extracting  test_stg2/image_00001.jpg
Extracting  test_stg2/image_00002.jpg
Extracting  test_stg2/image_00003.jpg
Extracting  test_stg2/image_00004.jpg
Extracting  test_stg2/image_00005.jpg
Extracting  test_stg2/image_00006.jpg
Extracting  test_stg2/image_00007.jpg
Extracting  test_stg2/image_00008.jpg
Extracting  test_stg2/image_00009.jpg
Extracting  test_stg2/image_00010.jpg
Extracting  test_stg2/image_00011.jpg
Extracting  test_stg2/image_00012.jpg
Extracting  test_stg2/image_00013.jpg
Extracting  test_stg2/image_00014.jpg
Extracting  test_stg2/image_00015.jpg
Extracting  test_stg2/image_00016.jpg
Extracting  test_stg2/image_00017.jpg
Extracting  test_stg2/image_00018.jpg
Extracting  test_stg2/image_00019.jpg
Extracting  test_stg2/image_00020.jpg
Extracting  test_stg2/image_00021.jpg
Extracting  test_stg2/image_

In [14]:
!rm -rf __MACOSX/
!rm *.zip
!rm *.7z

In [26]:
# Quick Stats
%cd $train_dir
!ls | while read line; do echo "$line"=`ls "$line" | wc -l`; done

/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
ALB=1719
BET=200
DOL=117
LAG=67
NoF=465
OTHER=299
SHARK=176
YFT=734


In [27]:
# Total=
print(1719+200+117+67+465+299+176+734)

3777


In [47]:
# Create Validation
#num_valid_one_class = 60 # As unbalanced, let's take a percentage of each class (may e.g. oversample later)

def create_validation(class_name, sample_size):
    %cd $TRAIN_DIR
    %mkdir -p $VALID_DIR/$class_name
    g = glob(class_name+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(sample_size): os.rename(shuf[i], VALID_DIR + shuf[i])

In [48]:
# validation_percentage_size = 10%

create_validation("ALB", 172)
create_validation("BET", 20)
create_validation("DOL", 12)
create_validation("LAG", 7)
create_validation("NoF", 47)
create_validation("OTHER", 30)
create_validation("SHARK", 18)
create_validation("YFT", 74)

/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train


In [49]:
# Create Sample (only train as validation set quite small anyway)
def create_train_sample(class_name, sample_size):
    %cd $TRAIN_DIR/
    %mkdir -p $TRAIN_SAMPLE_DIR/$class_name
    g = glob(class_name+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(sample_size): copyfile(shuf[i], TRAIN_SAMPLE_DIR + shuf[i])

In [51]:
# Quick Stats (having removed some into validation)
%cd $train_dir
!ls | while read line; do echo "$line"=`ls "$line" | wc -l`; done

/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
ALB=1547
BET=180
DOL=105
LAG=60
NoF=418
OTHER=269
SHARK=158
YFT=660


In [54]:
# train_sample_percentage_size = 10%

create_train_sample("ALB", 155)
create_train_sample("BET", 18)
create_train_sample("DOL", 11)
create_train_sample("LAG", 6)
create_train_sample("NoF", 42)
create_train_sample("OTHER", 27)
create_train_sample("SHARK", 17)
create_train_sample("YFT", 66)

/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train
/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/train


In [55]:
# Create single 'unknown' class for test set
%mv $TEST_STG1_DIR/*.jpg $TEST_STG1_DIR/unknown/
%mv $TEST_STG2_DIR/*.jpg $TEST_STG2_DIR/unknown/

mv: cannot stat ‘/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring//test_stg1//*.jpg’: No such file or directory
mv: cannot stat ‘/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring//test_stg2//*.jpg’: No such file or directory


# 2. Finetune model on full data (not sample as small data)

In [3]:
def fit_generator_helper(model, result_dir_name, lr=0.1, nb_epoch=1):
    K.set_value(model.optimizer.lr, lr)
    
    now = datetime.now().strftime("%Y%m%d_%H%M%S.h5")
    results_dir = RESULTS_DIR + "/" + result_dir_name + "/"
    distutils.dir_util.mkpath(results_dir)
    
    model.fit_generator(trn_batches,
                        samples_per_epoch=trn_batches.nb_sample,
                        nb_epoch=nb_epoch,
                        validation_data=val_batches,
                        nb_val_samples=val_batches.nb_sample,
                        callbacks=[CSVLogger(results_dir+"epoch_results.csv", separator=',', append=True)])
    model.save_weights(results_dir + now )
    return model

def fit_precomputed_helper(model, result_dir_name, lr=0.1, nb_epoch=1):  
    K.set_value(model.optimizer.lr, lr)
    
    now = datetime.now().strftime("%Y%m%d_%H%M%S.h5")
    results_dir = RESULTS_DIR + "/" + result_dir_name + "/"
    distutils.dir_util.mkpath(results_dir)
    
    model.fit(trn_conv_features, trn_labels,
              batch_size=batch_size, 
              nb_epoch=nb_epoch,
              validation_data=(val_conv_features, val_labels),
              shuffle=True, 
              callbacks=[CSVLogger(results_dir+"epoch_results.csv", separator=',', append=True)])
    model.save_weights(results_dir + now)
    return model

def predict_generator_helper(model, batches):
    # Be sure get_batches has used shuffle=False
    return model.predict_generator(batches,
                                   batches.nb_sample)

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

In [4]:
# Create base model
resnet_base = ResNet50(include_top=False, weights='imagenet')

#classifier_input_shape = resnet_base.layers[-1].output_shape[1:] # i.e. shape of conv features (produces (None, None, None, 2048))
classifier_input_shape = (1, 1, 2048)
classifier_input = Input(shape=classifier_input_shape)

### Precompute convolutional output (to save training time)

In [5]:
def preprocess_input(x, data_format=None):
    # Based on function at https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py
    # Altered for 3D input, assumes data_format='channels_last'
    
    """Preprocesses a tensor encoding a batch of images.
    # Arguments
        x: input Numpy tensor, 3D.
        data_format: data format of the image tensor.
    # Returns
        Preprocessed tensor.
    """
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

# Create (Raw Image) Batches
def get_batches(path,
                gen=ImageDataGenerator(preprocessing_function=preprocess_input),
                shuffle=True,
                batch_size=64,
                target_size=(224,224),
                class_mode='categorical'):
    return gen.flow_from_directory(path, 
                                   target_size=target_size,
                                   class_mode=class_mode,
                                   shuffle=shuffle, 
                                   batch_size=batch_size)

def create_precomputed_data(model, batches):
    filenames = batches.filenames
    conv_features = predict_generator_helper(model, batches)
    labels = one_hot(batches.classes)
    return (filenames, conv_features, labels)

def get_batch_data(batches):
    return np.concatenate([batches.next() for i in range(batches.nb_sample)])

In [6]:
batch_size = 32 # multiple of 2 and as large as you can (encourages more stable training), but no larger than 64.

In [7]:
trn_batches = get_batches(TRAIN_DIR, batch_size=batch_size, shuffle=False)
val_batches = get_batches(VALID_DIR, batch_size=batch_size, shuffle=False)

trn_filenames, trn_conv_features, trn_labels = create_precomputed_data(resnet_base, trn_batches)
val_filenames, val_conv_features, val_labels = create_precomputed_data(resnet_base, val_batches)

Found 3397 images belonging to 8 classes.
Found 380 images belonging to 8 classes.


In [6]:
%mkdir -p $RESULTS_DIR/ResNet50_conv_feats

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def save_precomputed_data(filenames, conv_feats, labels, features_base_name="ResNet50_conv_feats/trn_"):
    save_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat', np.array(filenames))
    save_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat', conv_feats)
    save_array(RESULTS_DIR+"/"+features_base_name+'labels.dat', np.array(labels))
    
save_precomputed_data(trn_filenames, trn_conv_features, trn_labels, "ResNet50_conv_feats/trn_")
save_precomputed_data(val_filenames, val_conv_features, val_labels, "ResNet50_conv_feats/val_")

NameError: name 'trn_filenames' is not defined

### Run finetuning

In [7]:
def load_array(fname):
    return bcolz.open(fname)[:]

def load_precomputed_data(features_base_name="ResNet50_conv_feats/trn_"):
    filenames = load_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat').tolist()
    conv_feats = load_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat')
    labels = load_array(RESULTS_DIR+"/"+features_base_name+'labels.dat')
    return filenames, conv_feats, labels
                          
trn_filenames, trn_conv_features, trn_labels = load_precomputed_data("ResNet50_conv_feats/trn_")
val_filenames, val_conv_features, val_labels = load_precomputed_data("ResNet50_conv_feats/val_")

assert len(trn_filenames) == 3397, "trn_filenames not as expected"
assert trn_conv_features.shape == (3397, 1, 1, 2048), "trn_conv_features not as expected"
assert trn_labels.shape == (3397, 8), "trn_labels not as expected"

assert len(val_filenames) == 380, "val_filenames not as expected"
assert val_conv_features.shape == (380, 1, 1, 2048), "val_conv_features not as expected"
assert val_labels.shape == (380, 8), "val_labels not as expected"

### Multi-Output

The images indicate that there is often a lot of noise in the images. Fortunately a kaggler has been through the training images and manually annotated the fish: https://www.kaggle.com/c/the-nature-conservancy-fisheries-monitoring/discussion/25902

In [104]:
%cd $TRAIN_DIR
%mkdir manual_annotations
%cd manual_annotations
!wget https://kaggle2.blob.core.windows.net/forum-message-attachments/147332/5471/other_labels.json
!wget https://kaggle2.blob.core.windows.net/forum-message-attachments/158691/5864/NoF_labels.json
!wget https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/5458/bet_labels.json
!wget https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/5459/shark_labels.json
!wget https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/5460/dol_labels.json
!wget https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/5461/yft_labels.json
!wget https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/5462/alb_labels.json
!wget https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/5463/lag_labels.json

--2017-05-12 13:42:33--  https://kaggle2.blob.core.windows.net/forum-message-attachments/158691/5864/NoF_labels.json
Resolving kaggle2.blob.core.windows.net (kaggle2.blob.core.windows.net)... 40.116.120.24
Connecting to kaggle2.blob.core.windows.net (kaggle2.blob.core.windows.net)|40.116.120.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 333927 (326K) [application/json]
Saving to: ‘NoF_labels.json’

100%[======================================>] 333,927      914KB/s   in 0.4s   

2017-05-12 13:42:34 (914 KB/s) - ‘NoF_labels.json’ saved [333927/333927]

--2017-05-12 13:42:35--  https://kaggle2.blob.core.windows.net/forum-message-attachments/147157/5458/bet_labels.json
Resolving kaggle2.blob.core.windows.net (kaggle2.blob.core.windows.net)... 40.116.120.24
Connecting to kaggle2.blob.core.windows.net (kaggle2.blob.core.windows.net)|40.116.120.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82471 (81K) [application/json]
Saving to: 

In [8]:
anno_classes = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft', 'NoF']

LABELS_DIR = DATA_ROOT_DIR + "/manual_annotations"

bb_json = {}
for c in anno_classes:
    j = json.load(open(LABELS_DIR+'/{}_labels.json'.format(c), 'r'))
    for one_file in j:
        if 'annotations' in one_file.keys() and len(one_file['annotations'])>0:
            bb_json[one_file['filename'].split('/')[-1]] = sorted(
                one_file['annotations'], key=lambda x: x['height']*x['width'])[-1]
            
bb_json['img_04908.jpg']

{'class': 'rect',
 'height': 246.75000000000074,
 'width': 432.8700000000013,
 'x': 465.3000000000014,
 'y': 496.32000000000147}

In [9]:
# For any images that have no annotations, we'll create an empty bounding box.

empty_bbox = {'height': 0., 'width': 0., 'x': 0., 'y': 0.}

print(len(bb_json))

raw_trn_filenames = [f.split("/")[-1] for f in trn_filenames]
raw_val_filenames = [f.split("/")[-1] for f in val_filenames]

for f in raw_trn_filenames:
    if f not in bb_json.keys(): bb_json[f] = empty_bbox
for f in raw_val_filenames:
    if not f in bb_json.keys(): bb_json[f] = empty_bbox
        
print(len(bb_json))
bb_json

3762
3777


{'img_00928.jpg': {'class': 'rect',
  'height': 92.00000000000131,
  'width': 159.00000000000227,
  'x': 696.0000000000095,
  'y': 348.0000000000048},
 'img_03025.jpg': {'class': 'rect',
  'height': 118.44000000000003,
  'width': 300.33000000000004,
  'x': 195.99000000000004,
  'y': 73.32000000000002},
 'img_07359.jpg': {'class': 'rect',
  'height': 40.0,
  'width': 155.0,
  'x': 435.0,
  'y': 485.0},
 'img_01921.jpg': {'class': 'rect',
  'height': 80.00443250132031,
  'width': 161.6931688447737,
  'x': 781.5169827497397,
  'y': 300.64823582075115},
 'img_05490.jpg': {'class': 'rect',
  'height': 199.0,
  'width': 85.0,
  'x': 115.0,
  'y': 481.0},
 'img_02377.jpg': {'class': 'rect',
  'height': 180.0,
  'width': 236.0,
  'x': 380.0,
  'y': 42.0},
 'img_03267.jpg': {'class': 'rect',
  'height': 112.0,
  'width': 496.0,
  'x': 596.0,
  'y': 225.0},
 'img_03141.jpg': {'class': 'rect',
  'height': 186.1200000000019,
  'width': 101.520000000001,
  'x': 569.6400000000059,
  'y': 250.9800000

In [10]:
# Finally, we convert the dictionary into an array, and convert the coordinates to our resized 224x224 images.

bb_params = ['height', 'width', 'x', 'y']

def convert_bb(bb, size):
    bb = [bb[p] for p in bb_params]
    conv_x = (224. / size[0])
    conv_y = (224. / size[1])
    bb[0] = bb[0]*conv_y
    bb[1] = bb[1]*conv_x
    bb[2] = max(bb[2]*conv_x, 0)
    bb[3] = max(bb[3]*conv_y, 0)
    return bb

trn_sizes = [PIL.Image.open(TRAIN_DIR+f).size for f in trn_filenames]
trn_bbox = np.stack([convert_bb(bb_json[f], s) for f,s in zip(raw_trn_filenames, trn_sizes)], 
                   ).astype(np.float32)

val_sizes = [PIL.Image.open(VALID_DIR+f).size for f in val_filenames]
val_bbox = np.stack([convert_bb(bb_json[f], s) for f,s in zip(raw_val_filenames, val_sizes)], 
                   ).astype(np.float32)

In [11]:
def to_plot(img):
    # Assumes tensorflow backend
    return np.rollaxis(img, 0, 1).astype(np.uint8)

def plot(img):
    plt.imshow(to_plot(img))

def create_rect(bb, color='red'):
    return plt.Rectangle((bb[2], bb[3]), bb[1], bb[0], color=color, fill=False, lw=3)

def show_bb(i, bbox, data):
    bb = bbox[i]
    plot(data[i])
    plt.gca().add_patch(create_rect(bb))
    
def get_data(path, target_size=(224,224)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size, gen=ImageDataGenerator())
    return np.concatenate([batches.next() for i in range(batches.nb_sample)])

trn_data = get_data(TRAIN_DIR)
val_data = get_data(VALID_DIR)

Found 3397 images belonging to 8 classes.


AttributeError: 'DirectoryIterator' object has no attribute 'nb_sample'

In [12]:
show_bb(0, trn_bbox, trn_data)

NameError: name 'trn_data' is not defined

In [13]:
# Create classifier model
# (base on `In [23]` https://github.com/asmith26/courses/blob/master/deeplearning1/nbs/lesson7.ipynb)

p = 0.6

#x = BatchNormalization(axis=-1)(classifier_input)
#x = Flatten()(x)
x = Flatten()(classifier_input)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x_bb = Dense(4, name='bb')(x)
x_class = Dense(8, activation='softmax', name='class')(x)
                                                     
resnet50_v1 = Model([classifier_input], [x_bb, x_class])
resnet50_v1.compile(Adam(lr=0.01), 
                    loss=['mse', 'categorical_crossentropy'],
                    metrics=['accuracy'], 
                    loss_weights=[.001, 1.])

In [14]:
def fit_precomputed_helper_multi_output(model, result_dir_name, lr=0.1, nb_epoch=1):  
    K.set_value(model.optimizer.lr, lr)
    
    now = datetime.now().strftime("%Y%m%d_%H%M%S.h5")
    results_dir = RESULTS_DIR + "/" + result_dir_name + "/"
    distutils.dir_util.mkpath(results_dir)
    
    model.fit(trn_conv_features, [trn_bbox, trn_labels],
              batch_size=batch_size, 
              nb_epoch=nb_epoch,
              validation_data=(val_conv_features, [val_bbox, val_labels]),
              shuffle=True, 
              callbacks=[CSVLogger(results_dir+"epoch_results.csv", separator=',', append=True)])
    model.save_weights(results_dir + now)
    return model

In [24]:
resnet50_v1.summary()
#from keras.utils.visualize_util import plot
#from keras.utils import plot_model
from keras.utils import model_to_dot

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 1, 1, 2048)    0                                            
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 2048)          0           input_2[0][0]                    
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 512)           1049088     flatten_1[0][0]                  
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 512)           2048        dense_1[0][0]                    
___________________________________________________________________________________________

ImportError: cannot import name 'model_to_dot'

In [23]:
resnet50_v1 = fit_precomputed_helper_multi_output(resnet50_v1, "resnet50_v1", lr=0.01, nb_epoch=5)
# Ran 5 times

NameError: name 'batch_size' is not defined

In [ ]:
def do_clip(arr, mx):
    # 
    clipped = np.clip(arr, (1-mx)/1, mx)
    return clipped/clipped.sum(axis=1)[:, np.newaxis]


### Data Augementation with classifier_model_v2

In [ ]:
We can't work from the precomputed

In [ ]:
# Create base model
resnet_base = ResNet50(include_top=False, weights='imagenet')

#classifier_input_shape = resnet_base.layers[-1].output_shape[1:] # i.e. shape of conv features (produces (None, None, None, 2048))
classifier_input_shape = (1, 1, 2048)
classifier_input = Input(shape=classifier_input_shape)

# Create classifier model
# (base on `In [23]` https://github.com/asmith26/courses/blob/master/deeplearning1/nbs/lesson7.ipynb)

p = 0.6

x = Flatten()(classifier_input)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x = Dense(8, activation='softmax')(x)
                                                     
resnet50_v1 = Model(classifier_input, x)
resnet50_v1.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
classifier_model_v2_0_2 = fit_precomputed_helper(classifier_model_v2_0_2, "classifier_model_v2_0_2", lr=0.001, nb_epoch=10)
# Ran 5 times

# 3. Generate and Validate Predictions